## Logistic Matrix Factorization

Maximize the following :

$$
\mathcal{L}(W, H) = \sum_{i=1}^I \sum_{j=1}^J M(i,j) ( Y(i,j) \log (\sigma(W(i) H(j))) + (1 - Y(i,j)) \log(1 - \sigma(W(i) H(j))) )
$$

Observed $I\times J$ binary matrix with possibly missing entries
$Y(i,j) \in \{0,1\}$

Mask Matrix
$M(i,j) = 1$ if $Y(i,j)$ is observed, $M(i,j) = 0$ if $Y(i,j)$ is not observed


Here:

$\sigma(x)$ is the sigmoid function defined as
\begin{eqnarray}
\sigma(x) & = & \frac{1}{1+e^{-x}}
\end{eqnarray}


### Properties of the sigmoid function
Note that

\begin{eqnarray}
\sigma(x) & = & \frac{e^x}{(1+e^{-x})e^x} = \frac{e^x}{1+e^{x}} \\
1 - \sigma(x) & = & 1 - \frac{e^x}{1+e^{x}} = \frac{1+e^{x} - e^x}{1+e^{x}} = \frac{1}{1+e^{x}}
\end{eqnarray}

\begin{eqnarray}
\sigma'(x) & = & \frac{e^x(1+e^{x}) - e^{x} e^x}{(1+e^{x})^2} = \frac{e^x}{1+e^{x}}\frac{1}{1+e^{x}} = \sigma(x) (1-\sigma(x))
\end{eqnarray}

\begin{eqnarray}
\log \sigma(x) & = & -\log(1+e^{-x}) = x - \log(1+e^{x}) \\
\log(1 - \sigma(x)) & = &  -\log({1+e^{x}})
\end{eqnarray}

#### Evaluating the gradient 

$$
\frac{d\mathcal{L}(W,H)}{dW(i)} = \sum_{j=1}^J (M(i,j) (Y(i,j) -\sigma(W(i) H(j)))) H(j)
$$

$$
\frac{d\mathcal{L}(W,H)}{dH(j)} = \sum_{i=1}^I  W(i) (M(i,j) (Y(i,j) -\sigma(W(i) H(j))))
$$


Then use alternating gradient descent 


In [19]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pylab as plt
import math


# Generate a random logistic regression problem

def sigmoid(t):
    return np.exp(t)/(1+np.exp(t))



I = 5
J = 10

# Random Mask 
M = np.random.rand(I,J)<0.8

# Random Parameters
W = np.random.randn(I,1)
H = np.random.randn(1,J)

Y = np.zeros((I,J))
# Generate class labels
pi = sigmoid(W*H)
#pid = sigmoidd(W*H)

for i in range(I):
    for j in range(J):
        if not M[i,j]:
            Y[i,j] = np.nan
        else:
            Y[i,j] = 1 if pi[i,j] < np.random.rand() else 0
            

#Solution starts here


print('W and H used to generate problem matrix: W=' , W, ', H =', H)



def deriveH(i,j):
    sum = 0
    for it in range(I):
        m = 1
        if M[it][j] is None:
            m = 0
        tmp = W[it]*((m*Y[it][j]) - sigmoid(W[it]*H[0][j]))
        if np.isnan(tmp):
            tmp = 0
        sum = sum + tmp
    return sum
        

#def deriveW(i,j):
#    if not M[i,j]:
#        return 0
#    else:
#        (H[0][j]*(((Y[i,j]-1)*np.exp(W[i]*H[0][j]))+Y[i,j]))/(np.exp(H[0][j]*W[i]) + 1)

def deriveW(i,j):
    sum = 0
    for jt in range(J):
        m = 1
        if M[i][jt] is None:
            m = 0
        tmp = ((m*Y[i][jt]) - (sigmoid(W[i]*H[0][jt])))*H[0][jt]
        if np.isnan(tmp):
            tmp = 0
        sum = sum + tmp
    return sum


#for i in range(I):
#    for j in range(J):
#        print(deriveH(i,j))
#        print(deriveW(i,j))


print('===============================')

#Assign new randoms to start gradient
Wi = W
Hi = H
W = np.random.randn(I,1)
H = np.random.randn(1,J)


trials = 1
epoch = 50
eta = 0.0003366
for t in range(trials):
    for e in range(epoch):
        for i in range(I):
            for j in range(J):
                derW = deriveW(i,j)
                if(derW is not None):
                    #print('Wi = ' , Wi[i] , 'W = ' , W[i] ,'derivative = ', derW , ' value change:', (-(eta*derW)))
                    W[i] = W[i] - (eta*derW)
                derH = deriveH(i,j)
                if(derH is not None):
                    H[0][j] = H[0][j] - (eta*derH)
                #derW = deriveW(i,j)
                    #if(derW is not None):
                        #print('Wi = ' , Wi[i] , 'W = ' , W[i] ,'derivative = ', derW , ' value change:', (-(eta*derW)))
                        #W[i] = W[i] - (eta*derW)
                    #derH = deriveH(i,j)
                    #if(derH is not None):
                        #H[0][j] = H[0][j] - (eta*derH)
                #else:
                    #derW = deriveW(i,j)
                    #if(derW is not None):
                        #print('Wi = ' , Wi[i] , 'W = ' , W[i] ,'derivative = ', derW , ' value change:', (-(eta*derW)))
                        #W[i] = W[i] - (eta*derW)
                    #derH = deriveH(i,j)
                    #if(derH is not None):
                        #H[0][j] = H[0][j] - (eta*derH)

print('W and H found after gradient solution: W=' , W, ', H =', H)

W and H used to generate problem matrix: W= [[-0.87062417]
 [-0.79065176]
 [ 0.77778248]
 [ 0.18022026]
 [ 0.88964628]] , H = [[ 0.98643023 -0.4016104   0.36099611 -0.69621904 -0.47657641 -0.35771474
  -0.83280653 -0.03511777 -0.2451144  -0.9655413 ]]
W and H found after gradient solution: W= [[-0.64535762]
 [-0.74629652]
 [-0.23971459]
 [ 0.44929422]
 [-0.84119892]] , H = [[ 0.14874123 -1.18787148 -0.3098156  -0.11798829 -1.33497689 -0.88957889
  -0.46182509 -0.62079179  0.1958358  -0.67291302]]


Task: 
Given $Y$ and $M$ only find a good $W$ and $H$ by maximizing the objective $\mathcal{L}$